In [1]:
import torch
import numpy as np

from scipy.interpolate import RegularGridInterpolator
from dataset_2 import get_rays
from ml_helpers import testing

# load NeRF model trained in my coral_nerf code
model_pth = torch.load('model_training.pth')

In [2]:
# Load test rays of dataset for each test image and pose

test_o, test_d, test_target_px_values = get_rays('fox', mode='test') # getting test rays

### Testing out NeRFStudio API way of representing a grid
However it appears that to properly use it would require that my NeRF be trained using the NeRFStudio pipeline API
To use NeRFStudio:

!pip install nerfstudio

In [3]:
from nerfstudio.field_components.encodings import HashEncoding
from nerfstudio.field_components.spatial_distortions import SpatialDistortion # maybe I can simply try to distort a sample?

lod: int = 8
device = 'cpu'

ahessian = torch.zeros(((2**lod)+1)**3) #.to(device)
deform_field = HashEncoding(num_levels = 1, 
                            min_res = 2**lod,
                            max_res = 2**lod,
                            log2_hashmap_size = lod*3+1, #simple regular grid (hash table size > grid size)
                            features_per_level = 3,
                            hash_init_scale = 0.,
                            implementation = "torch",
                            interpolation = "Linear")
deform_field.scalings = torch.tensor([2**lod])

x = SpatialDistortion.forward(deform_field, positions=[7,4,3]) # not sure how to actually define deformation values in the deformation field...
print(x)

None


### Other approach to deformation field: Using meshgrid, with a vector [x,y,z] defined at each point

In [4]:
# Define grid dimensions
M = 128
x_ = np.arange(0, M)
y_ = np.arange(0, M)
z_ = np.arange(0, M)

# Create meshgrid
X, Y, Z = np.meshgrid(x_, y_, z_, indexing='ij')

# initalize the grid with .01's (trying to define with single values for now)
values = np.ones_like(X)*0.01
interp = RegularGridInterpolator((x_, y_, z_), values) # trilinear interpolation function from scipy for getting values between vertices

## Approximating the covariance matrix

Function for approximating the hessian from which the covariance can be found. Currently based on the BayesRays approach, still incomplete.

In [5]:
# .find_uncertainty(points, offsets, outputs['rgb'], pipeline.model.field.spatial_distortion

def find_uncertainty(points, deform_points, rgb, distortion):
    '''
    Approximates the hessian matrix used to find the covariance of the uncertainty field.

    Arguments:
        - points
        - rgb (pixel values in the output)
    Outputs:
        - Approximated Hessian matrix
    '''
    colors = torch.sum(rgb, dim=0) # sum all the colors along the ray

    # Get r,g,and b channel from deform points
    colors[0].backward(retain_graph=True)
    r = deform_points.grad.clone().detach().view(-1,3)
    deform_points.grad.zero_()
    colors[1].backward(retain_graph=True)
    g = deform_points.grad.clone().detach().view(-1,3)
    deform_points.grad.zero_()
    colors[2].backward()
    b = deform_points.grad.clone().detach().view(-1,3)
    deform_points.grad.zero_()

    # continue...
    ...
    return ahessian

### Calling my rendering function with perturbed [x,y,z] camera positions for each ray to render approximate color for each pixel

This is still in progress, figuring out what modificiations to make to the rendering function to successfully add perturbation values

In [6]:
# render with the perturbations: test function calls the perturbed rendering function

img,_,psnr = testing(model=model_pth, o=torch.from_numpy(test_o[4]).to(device).float(), d=torch.from_numpy(test_d[4]).to(device).float(),
                tn=8., tf=12., nb_bins=100, chunk_size=60, H=400, W=400,target=test_target_px_values[4].reshape(400,400,3)
                ,interp=interp) # interp added to pass interpolation object into the rendering function

TypeError: expected np.ndarray (got Tensor)